# Plot spatial properties

In this notebook we provide code to show the spatial distribution of extracted properties of cells using Napri

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%gui qt

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
matplotlib.rc("figure", figsize=(10,5))

import seaborn as sns

import pathlib
import napari

import h5py


## Define function

Below we define the function to create a spatial map of a property of choice

In [ ]:
def plot_spatial_map(label_stack, df, property, frame):
    """
    Plot spatial map of cell property
    
    Parameters:
        label_stack: Numpy ND array [t,y,x] with label image stack
        df: pandas data frame of lineage object
        property: key of cell property contained in lineage object
        frame: frame to show
        
    Returns:
        spatial_map: Numpy ND array [y,x] spatial map of cell property at given frame
    """

    #create color map where Nan is shown as black  
    colMap = cm.get_cmap("viridis").copy()
    colMap.set_bad(color='black')
    
    # get label image:
    labels = label_stack[frame,:,:] #assumes frame is the first dimension
    spatial_map = np.full(labels.shape, np.nan)
    
    # Go over cells in selected frame:
    for cnb in np.unique(labels):
        if cnb==0: continue
        #assign cells mask area the phenotype of choice
        spatial_map[labels==cnb] = df.loc[(df['frame']==frame) & (df['labelID']==cnb),property].item()
    
    return spatial_map

## Load Data
We load the data from the last notebook;

In [4]:
#load images and labels
with h5py.File(pos_file, 'r') as f:
    labels = f['labels'][:]
    images = f['images'][:]
    
df = pd.read_csv(lineage_file)    

## Plot data

you can set `property_to_show` to any column name in the dataframe

In [5]:
property_to_show = 'id_colony'
#loop over all frames
props = [plot_spatial_map(labels, df, property_to_show, frame=fr) for fr in range(labels.shape[0])]
propstack = np.stack(props) #convert list of arrays to array

Now we create a Napari window adding the property to show

In [6]:
viewer = napari.view_image(images)
viewer.add_labels(labels)
viewer.add_image(propstack)
viewer.show()